In [115]:
import pandas as pd
from pathlib import Path
import plotly as plt
from datetime import datetime as dt
from sqlalchemy import create_engine,inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float 
from sqlalchemy.orm import Session


In [116]:
master_df = pd.DataFrame()

In [117]:
def Import_clean_dataset(df,month,year,master_df):
    
    #Format imported dataframe
    df = df.drop(labels=0)
    df = df.reset_index()

    df = df.rename(columns={'Unnamed: 1':'item','Unnamed: 3':'price','Unnamed: 4':'sold', 'Unnamed: 7':'tot_value'})
    df = df[['item','price','sold','tot_value']]
    
    df = df.drop_duplicates()
    df = df.dropna()

    df['item'] = df['item'].str.strip()
    df['sold'] = df['sold'].apply(pd.to_numeric, errors='coerce')
    df['price'] = df['price'].apply(pd.to_numeric, errors='coerce')
    df['tot_value'] = df['tot_value'].str.replace(r',', '')
    df['tot_value'] = df['tot_value'].apply(pd.to_numeric, errors='coerce')
    df['month'] = month
    df['month'] = df['month'].astype(float)
    df['year'] = year
    df['year'] = df['year'].astype(float)
    
    #Create Date Column - conditional statement: formatting month input to follow '00' style
    if month < 10:
        df['date'] = str(20)+str(year)+'-'+ str(0)+str(month)+'-'+'01'
    else:
        df['date'] = str(20)+str(year) + '-'+str(month)+'-'+'01'

    # Append df to master_df    
    master_df = master_df.append(df, ignore_index=True)
    
    return (master_df)

Set time range

In [118]:
Starting_year = 17
Ending_year = 22

Import all Files

In [119]:
# Loop to through range of the year time period 
for year in range(Starting_year, Ending_year+1):
    main = 'Resources/POSdata/'
    
    # Loop through range of months within a year (12)
    for month in range(1,13):
        
        # Conditional loop to allow for '00' month formatting
        if month >=10:
            file = main + str(year) + "-" + str(month) + '.csv'
        else:
            file = main + str(year) + "-" + str(0) + str(month) + '.csv'
        print(f'The file: {file} has been loaded')
        # Read in csv file to df from the filename created
        df = pd.read_csv(file)

        # Clean dataframe before adding it to master_df
        master_df = Import_clean_dataset(df,month,year,master_df)
        

The file: Resources/POSdata/17-01.csv has been loaded
The file: Resources/POSdata/17-02.csv has been loaded
The file: Resources/POSdata/17-03.csv has been loaded
The file: Resources/POSdata/17-04.csv has been loaded
The file: Resources/POSdata/17-05.csv has been loaded
The file: Resources/POSdata/17-06.csv has been loaded
The file: Resources/POSdata/17-07.csv has been loaded
The file: Resources/POSdata/17-08.csv has been loaded
The file: Resources/POSdata/17-09.csv has been loaded
The file: Resources/POSdata/17-10.csv has been loaded
The file: Resources/POSdata/17-11.csv has been loaded
The file: Resources/POSdata/17-12.csv has been loaded
The file: Resources/POSdata/18-01.csv has been loaded
The file: Resources/POSdata/18-02.csv has been loaded
The file: Resources/POSdata/18-03.csv has been loaded
The file: Resources/POSdata/18-04.csv has been loaded
The file: Resources/POSdata/18-05.csv has been loaded
The file: Resources/POSdata/18-06.csv has been loaded
The file: Resources/POSdata/

Explore master_df

In [120]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8625 entries, 0 to 8624
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   item       8625 non-null   object 
 1   price      8625 non-null   float64
 2   sold       8625 non-null   float64
 3   tot_value  8625 non-null   float64
 4   month      8625 non-null   float64
 5   year       8625 non-null   float64
 6   date       8625 non-null   object 
dtypes: float64(5), object(2)
memory usage: 471.8+ KB


In [121]:
master_df['item'].value_counts().sort_values(ascending=False).head(60)

Plain Naan            73
Garlic Naan           73
Palak Paneer          73
Curry Puffs           72
White Rice Large      72
Mustard Chicken       72
Onion Bhajia          72
chicken sticks        72
Samosas - Vege        72
chicken strips        72
Butter Chicken        72
Madras Chicken        72
Chicken Korma         72
CH Tikka Masala       72
Curry Samosas         72
Beef Vindaloo         72
Mini Spring Rolls     72
upgrade ONE SIZE      72
Chefs SP Chicken.     72
Large Samosas         72
Can drinks            72
Madras Lamb           72
Lamb Korma            72
Lamb Vindaloo         72
Chefs SP Lamb         72
Chk In Coconut        72
Rogan Josh            72
Madras Beef           72
Vege Korma            72
Dhal                  72
Beef Korma            72
Chefs Sp beef         72
Mango Chutney         72
Raita                 72
K Praw N Coconut      72
Pappodums x 4         72
Parathas              72
Chapattis             72
Kukulmas Curry        72
Yellow Rice Large     72


In [122]:
master_df.head()

,item,price,sold,tot_value,month,year,date
0,Curry Puffs,7.0,22.0,147.0,1.0,17.0,2017-01-01
1,Large Samosas,6.5,18.0,121.5,1.0,17.0,2017-01-01
2,Samosas - Vege,7.5,19.0,142.5,1.0,17.0,2017-01-01
3,Mini Spring Rolls,7.5,20.0,150.0,1.0,17.0,2017-01-01
4,Curry Samosas,4.0,19.0,85.0,1.0,17.0,2017-01-01


In [123]:
master_df['item'] = master_df['item'].astype(str)
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8625 entries, 0 to 8624
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   item       8625 non-null   object 
 1   price      8625 non-null   float64
 2   sold       8625 non-null   float64
 3   tot_value  8625 non-null   float64
 4   month      8625 non-null   float64
 5   year       8625 non-null   float64
 6   date       8625 non-null   object 
dtypes: float64(5), object(2)
memory usage: 471.8+ KB


Create and load SQLite dataframe

In [124]:
Base = declarative_base()

POS sales data table creation

In [125]:
class sales(Base):
    __tablename__ = 'sales'
    id = Column(Integer, primary_key=True)
    item = Column(String(255))
    price = Column(Integer)
    sold = Column(Integer)
    tot_value = Column(Float)
    month = Column(Integer)
    year = Column(Integer)
    date = Column(String(255))

Create engine with SQLite database file location

In [126]:
engine = create_engine("sqlite:///RestaurantDB.sqlite")

In [127]:
conn = engine.connect()

In [128]:
Base.metadata.create_all(engine)

In [129]:

session = Session(bind=engine)

Load and commit each row of the master_df into an sql table row

In [130]:
for row in range(len(master_df)):
    
    sqldata = sales(item = master_df.loc[row,'item'],price = master_df.loc[row,'price'], sold = master_df.loc[row,'sold'],
    tot_value=master_df.loc[row,'tot_value'], month = master_df.loc[row,'month'], year = master_df.loc[row,'year'], date = master_df.loc[row,'date'] )
    session.add(sqldata)

In [131]:
session.commit()

Check table for data entry

In [132]:
inspector = inspect(engine)

In [133]:
inspector.get_table_names()

['sales']

In [134]:
engine.execute('SELECT * FROM sales').fetchall()

[(1, 'Curry Puffs', 7, 22, 147.0, 1, 17, '2017-01-01'),
 (2, 'Large Samosas', 6.5, 18, 121.5, 1, 17, '2017-01-01'),
 (3, 'Samosas - Vege', 7.5, 19, 142.5, 1, 17, '2017-01-01'),
 (4, 'Mini Spring Rolls', 7.5, 20, 150.0, 1, 17, '2017-01-01'),
 (5, 'Curry Samosas', 4, 19, 85.0, 1, 17, '2017-01-01'),
 (6, 'Onion Bhajia', 5.5, 55, 325.5, 1, 17, '2017-01-01'),
 (7, 'chicken sticks', 9, 17, 142.5, 1, 17, '2017-01-01'),
 (8, 'Tikka Chicken', 9, 5, 42.0, 1, 17, '2017-01-01'),
 (9, 'chicken strips', 9, 15, 130.0, 1, 17, '2017-01-01'),
 (10, 'Butter Chicken', 16.5, 236, 4007.0, 1, 17, '2017-01-01'),
 (11, 'Madras Chicken', 16, 9, 150.0, 1, 17, '2017-01-01'),
 (12, 'Chicken Korma', 16.5, 34, 583.5, 1, 17, '2017-01-01'),
 (13, 'CH Tikka Masala', 16, 32, 534.0, 1, 17, '2017-01-01'),
 (14, 'Chicken Vindaloo', 18, 6, 100.0, 1, 17, '2017-01-01'),
 (15, 'Mustard Chicken', 18, 25, 422.0, 1, 17, '2017-01-01'),
 (16, 'Chk In Coconut', 16, 3, 50.0, 1, 17, '2017-01-01'),
 (17, 'Chicken Jalfrezi', 16, 8, 134.

In [135]:
rows = session.query(sales).count()
rows

8625

Close Connection

In [136]:
conn.close()

In [137]:
session.close()

Transform and load - Meat Historical Data

In [138]:
meat_master_df = pd.DataFrame()

Function to clean Meat Historical Data dataframe

In [139]:
def Import_clean_meat_df(df,item,meat_master_df):
    df = df.drop(labels=0)
    if item =='beef':
        df = df.rename(columns={'Beef - Monthly Price (Australian Dollar per Kilogram)': 'date', 'Unnamed: 1':'price', 'Unnamed: 2': 'percentage change'})
        print('beef')
    elif item == 'chicken':
        df = df.rename(columns={'Poultry (chicken) - Monthly Price (Australian Dollar per Kilogram)': 'date', 'Unnamed: 1':'price', 'Unnamed: 2': 'percentage change'})
        print('chicken')

    df['date'] = pd.to_datetime(df['date'], format="%b-%y")
    df['date'] = df['date'].dt.strftime("%b-%y")
    
    df['type'] = item

    df['percentage change'] = df['percentage change'].str.strip("%")
    df['percentage change'] = df['percentage change'].str.strip()
    
    df = df.drop(labels=1)
    
    df['percentage change'] = df['percentage change'].astype(float)
   
    df['price'] = df['price'].astype(float)
    
    meat_master_df = meat_master_df.append(df,ignore_index=True)
    
    return (meat_master_df)

Store file names

In [140]:
filename = ['beef', 'chicken']


Load filenames and import to dataframe using funtion to format dataframe and storing to meat_master_df

In [141]:
for files in filename:
    file = 'Resources/MeatPriceData/' + files + '.csv'
    df = pd.read_csv(file)
    meat_master_df = Import_clean_meat_df(df,files,meat_master_df)

beef
chicken


In [142]:
meat_master_df.head()

,date,price,percentage change,type
0,Dec-02,3.57,4.84,beef
1,Jan-03,3.50,-1.79,beef
2,Feb-03,3.35,-4.42,beef
3,Mar-03,3.35,0.14,beef
4,Apr-03,3.24,-3.55,beef


In [143]:
meat_master_df.tail()

,date,price,percentage change,type
471,May-22,5.25,5.83,chicken
472,Jun-22,5.22,-0.62,chicken
473,Jul-22,5.16,-1.16,chicken
474,Aug-22,4.77,-7.53,chicken
475,Sep-22,4.96,3.87,chicken


In [144]:
meat_master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476 entries, 0 to 475
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               476 non-null    object 
 1   price              476 non-null    float64
 2   percentage change  476 non-null    float64
 3   type               476 non-null    object 
dtypes: float64(2), object(2)
memory usage: 15.0+ KB


Create and Load SQlite database with Meat Historical data

In [145]:
Base = declarative_base()


Create meat_history table

In [146]:
class meat_history(Base):
    __tablename__ = "meat_historical_prices"
    id = Column(Integer, primary_key=True)
    date = Column(String(255))
    price = Column(Float)
    percentage_change = Column(Float)
    type = Column(String(255))
    
    

connect to sqlite database

In [147]:
engine = create_engine("sqlite:///RestaurantDB.sqlite")

In [148]:
conn = engine.connect()
Base.metadata.create_all(engine)
session = Session(bind=engine)


Load and commit each row of the meat_master_df into an sql table row

In [149]:
for row in range(len(meat_master_df)):
    
    sqldata = meat_history(date = meat_master_df.loc[row,'date'],price = meat_master_df.loc[row,'price'], percentage_change = meat_master_df.loc[row,'percentage change'],
    type=meat_master_df.loc[row,'type'])
    session.add(sqldata)

In [150]:

session.commit()



Inspect table for data

In [151]:
inspector = inspect(engine)

In [152]:

inspector.get_table_names()

['meat_historical_prices', 'sales']

In [153]:
engine.execute('SELECT * FROM meat_historical_prices').fetchall()

[(1, 'Dec-02', 3.57, 4.84, 'beef'),
 (2, 'Jan-03', 3.5, -1.79, 'beef'),
 (3, 'Feb-03', 3.35, -4.42, 'beef'),
 (4, 'Mar-03', 3.35, 0.14, 'beef'),
 (5, 'Apr-03', 3.24, -3.55, 'beef'),
 (6, 'May-03', 2.91, -10.16, 'beef'),
 (7, 'Jun-03', 2.82, -3.13, 'beef'),
 (8, 'Jul-03', 2.89, 2.49, 'beef'),
 (9, 'Aug-03', 3.24, 12.31, 'beef'),
 (10, 'Sep-03', 3.43, 5.95, 'beef'),
 (11, 'Oct-03', 3.36, -2.22, 'beef'),
 (12, 'Nov-03', 3.62, 7.77, 'beef'),
 (13, 'Dec-03', 3.45, -4.54, 'beef'),
 (14, 'Jan-04', 3.3, -4.42, 'beef'),
 (15, 'Feb-04', 3.07, -6.89, 'beef'),
 (16, 'Mar-04', 3.12, 1.55, 'beef'),
 (17, 'Apr-04', 3.27, 4.65, 'beef'),
 (18, 'May-04', 3.8, 16.42, 'beef'),
 (19, 'Jun-04', 3.98, 4.56, 'beef'),
 (20, 'Jul-04', 3.92, -1.34, 'beef'),
 (21, 'Aug-04', 4.11, 4.78, 'beef'),
 (22, 'Sep-04', 4.18, 1.65, 'beef'),
 (23, 'Oct-04', 3.81, -8.91, 'beef'),
 (24, 'Nov-04', 3.6, -5.46, 'beef'),
 (25, 'Dec-04', 3.53, -1.77, 'beef'),
 (26, 'Jan-05', 3.52, -0.47, 'beef'),
 (27, 'Feb-05', 3.46, -1.68, 'beef

In [156]:
rows = session.query(meat_history).count()
rows

476

In [154]:

conn.close()

session.close()